In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, roc_curve
import xgboost as xgb

In [2]:

train_data_path ="./../Data/train/train.csv"
test_data_path = "./../Data/test/test.csv"

dataset = pd.read_csv(train_data_path,skipinitialspace=True)
test_data = pd.read_csv(test_data_path,skipinitialspace=True)


train_dataset = dataset.sample(frac=0.7, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
        
dataset



,ID,Score,col1,col2,col3,col4,col5,col6,col7,col8,...,col3796,col3797,col3798,col3799,col3800,col3801,col3802,col3803,col3804,col3805
0,5,3.475628,0,4.058,0.824,0,10.267,0.728,4.403,0.050,...,1.067,0,0.0,0.115,30.395,24.541,0,0.415,0.997,0
1,8,3.601332,0,4.111,0.929,0,8.352,0.907,4.216,0.034,...,0.934,0,0.0,0.227,38.508,35.038,0,3.979,0.997,3
2,9,1.935003,0,4.139,0.833,66,9.494,0.733,4.069,0.267,...,1.722,0,0.0,0.148,27.932,19.518,0,0.849,0.999,0
3,12,3.283663,0,4.016,0.880,0,8.237,0.836,3.956,0.129,...,0.993,0,0.0,0.124,18.993,25.403,0,0.988,0.998,0
4,14,3.409121,0,4.657,0.522,0,35.882,0.383,4.234,-0.089,...,2.095,0,0.0,0.088,44.225,15.741,0,1.595,0.997,0
5,15,2.523486,0,4.623,0.711,66,35.891,0.581,4.154,0.327,...,3.785,0,0.0,0.078,157.893,20.733,0,3.691,0.997,0
6,18,0.919078,0,4.345,0.668,0,20.449,0.542,4.773,-0.013,...,1.604,1,0.0,0.107,5.552,14.887,0,4.480,0.996,4
7,19,2.769820,0,4.380,0.722,0,16.474,0.531,3.890,-0.056,...,1.292,0,0.0,0.090,36.866,18.200,0,3.190,0.998,0
8,20,2.681964,0,4.235,0.923,16,15.185,0.902,4.606,0.097,...,1.056,1,0.0,0.110,38.458,36.047,0,4.016,0.998,2
9,25,2.465977,0,4.214,0.522,66,3.885,0.426,4.156,0.346,...,1.726,1,0.0,0.109,0.000,9.349,0,0.411,0.999,0


In [3]:
# describe for Normalization
label = "Score"
train_stats = train_dataset.describe()
train_stats.pop(label)
train_stats = train_stats.transpose()

test_stats = test_dataset.describe()
test_stats.pop(label)
test_stats = test_stats.transpose()

test_data_stats = test_data.describe()
test_data_stats = test_data_stats.transpose()

# pop label
train_labels = train_dataset.pop(label)
test_labels = test_dataset.pop(label)

print(train_labels.shape)

(9612,)


In [4]:
# Normalization
def norm(train_dataset,train_stats):
    return (train_dataset - train_stats["mean"]) / train_stats["std"]

normed_train_data = norm(train_dataset,train_stats)
normed_test_data = norm(test_dataset,test_stats)
# normed_train_data = normed_train_data.dropna(axis=1)
# normed_test_data = normed_test_data.dropna(axis=1)

normed_train_data.fillna(0, inplace=True)
normed_test_data.fillna(0, inplace=True)
normed_test_data = norm(test_data,test_data_stats)
normed_test_data.fillna(0, inplace=True)


print(normed_train_data.shape)

(9612, 3806)


In [12]:
# 必要なライブラリのインポート
from sklearn.model_selection import GridSearchCV
# 動かすパラメータを明示的に表示
params = {"learning_rate":[0.1,0.3,0.5],
        "max_depth": [2,3,5,10],
         "subsample":[0.5,0.8,0.9,1],
         "colsample_bytree": [0.5,1.0],
         }
# モデルにインスタンス生成
mod = xgb.XGBRegressor()
# ハイパーパラメータ探索
cv = GridSearchCV(mod, params, cv = 3, scoring= 'r2', n_jobs =-1,verbose=2)

#　trainデータとtestデータに分割
X_train, X_test, y_train, y_test = train_test_split(normed_train_data, train_labels, test_size = 0.3, random_state = 0)


# 予測モデルを作成
cv.fit(X_train, y_train)

#予測    
y_train_pred = cv.predict(X_train)
y_test_pred = cv.predict(X_test)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 20.4min


KeyboardInterrupt: 